# Multimodal Transformers

## 12-4-3 Audio data
Foundations of deep learning の12.4.3 Audio dataの解説を行います。音とは、空気の圧力変動として信号波形の形で保存されます。信号波形を直接的にDLモデルの入力として用いることは可能ではありますが、実用上は前処理としてMel spectrogramに通す、ということが一般的に用いられます。
transformerは、特に音声分類のために用いられることが多いです。モデルの性能を評価するために、Audioset Datasetというデータセットがよく用いられます。これは、各音声波形に対し「車」「動物」「笑い声」といったラベルが割り振られており、何らかの音声波形を通すことによってそのクラスを予測するといったものになります。Transformer以前はmel spectrogramを画像として扱い、CNNの入力として利用するというアプローチがとられていました。Transformerの利点の一つとして長い時間での依存関係・相関をとらえることができるという特徴があり、これが音声データを扱う上で非常に重要になってきます。
ここでは、mel spectrogramはまず画像として扱われ、トークンという一つの単位として分類されます。ViT(Vision Transformers)と同様の仕組みで画像をパッチに分割し、それらを一列に並べます。この場合、256の長さに並び替えられます。位置情報がこれらトークンに追加され、TransformerのEncoder層に通されます。最後のtransformer層からの出力トークンは、線形層によって複号され、ソフトマックス活性化関数を通すことによって出力が得られます。これらすべてのモデルは、クロスエントロピー誤差関数を用いることによってEnd-to-Endに訓練を行うことができます。
以下は、これらの性能比較を行うためにCNNとTransformerモデルを実装したものです。注意として、これらの実験は再現不可能である可能性があります。ViTに関しては、1080Tiに1日程度計算させる（epoch=200）ことによって良いモデルを作成することができるという計算効率の良さは証明済ですが、本稿で紹介されているASTモデルに関しては超高性能なコンピュータを使用している可能性があり、再現の目途が立っていません。
しかし、紹介されている論文(Gong,Chung,and Glass,2021)を見る限りアーキテクチャはViTに酷似しており、データの取り扱いも類似のものが見て取れましたのでViTでも同様の研究は再現可能だと考え実行を行っております。この訓練フェーズはAlvisという北欧でのリモートコンピューティングサービスを使用して、V100に計算を実行させています。